In [1]:
!pip install transformers datasets

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

In [4]:
!wget https://www.gutenberg.org/cache/epub/11/pg11.txt -O sample_dataset.txt

--2025-06-19 18:00:44--  https://www.gutenberg.org/cache/epub/11/pg11.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174354 (170K) [text/plain]
Saving to: ‘sample_dataset.txt’

sample_dataset.txt  100%[===================>] 170.27K  --.-KB/s    in 0.1s    

2025-06-19 18:00:44 (1.45 MB/s) - ‘sample_dataset.txt’ saved [174354/174354]



In [5]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def load_dataset(file_path):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

dataset = load_dataset("sample_dataset.txt")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Increase if needed
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=192, training_loss=2.936961809794108, metrics={'train_runtime': 1314.4358, 'train_samples_per_second': 0.291, 'train_steps_per_second': 0.146, 'total_flos': 25018712064000.0, 'train_loss': 2.936961809794108, 'epoch': 1.0})

In [13]:

input_text = "Alice was beginning to get very tired"


import torch

input_ids = tokenizer.encode(input_text, return_tensors='pt')
attention_mask = torch.ones_like(input_ids)

output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=100,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Alice was beginning to get very tired of the conversation, and she began to think of something else.

“I’m going to go to the library,” said Alice, “and I�ll find out what it is.‘
The library was full of books, but the first thing she did was to look at the bookshelves, which were all in the same place. Alice had never seen such a large collection of things, so she went on
